In [35]:
import pandas as pd
import numpy as np
import os
import json
import datetime

In [36]:
#read processed dataframe
df = pd.read_csv(r"..\Data\processed_data\df_processed.csv")
df


,Category,Detail,Cost,Date,Month_period,WeekofYear,DayofWeek,Day_of_Week_Category,Month
0,Eating Out,Brunch,15.00,2022-07-25,Jul - Aug 22,30,0,Weekday,7
1,Shopping,Uniqlo,144.50,2022-07-25,Jul - Aug 22,30,0,Weekday,7
2,Hobbies,Salsa Class,130.00,2022-07-27,Jul - Aug 22,30,2,Weekday,7
3,Groceries,Groceries,6.78,2022-07-27,Jul - Aug 22,30,2,Weekday,7
4,Eating Out,Ditsch,2.00,2022-07-27,Jul - Aug 22,30,2,Weekday,7
...,...,...,...,...,...,...,...,...,...
324,Groceries,Groceries,8.26,2023-02-21,Jan - Feb 23,8,1,Weekday,2
325,Groceries,Groceries,27.25,2023-02-22,Jan - Feb 23,8,2,Weekday,2
326,Groceries,Groceries,13.69,2023-02-23,Jan - Feb 23,8,3,Weekday,2
327,Rent + Bills,Phone Norway,4.50,2023-02-24,Jan - Feb 23,8,4,Weekday,2


In [37]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 329 entries, 0 to 328
Data columns (total 9 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   Category              329 non-null    object 
 1   Detail                329 non-null    object 
 2   Cost                  329 non-null    float64
 3   Date                  329 non-null    object 
 4   Month_period          329 non-null    object 
 5   WeekofYear            329 non-null    int64  
 6   DayofWeek             329 non-null    int64  
 7   Day_of_Week_Category  329 non-null    object 
 8   Month                 329 non-null    int64  
dtypes: float64(1), int64(3), object(5)
memory usage: 23.3+ KB


In [38]:
#add datetime
df['Date'] = pd.to_datetime(df['Date'])

In [39]:
#adding weekday; Week referend
df["WeekofYear"] = df.Date.dt.week
df["DayofWeek"] = df.Date.dt.day_of_week
SumperWeekofYear = df.groupby(["Date","WeekofYear","DayofWeek","Category"])["Cost"].sum()
SumperWeekofYear
df_SumperWeekofYear = pd.DataFrame(SumperWeekofYear)
df_SumperWeekofYear
df_SumperWeekofYear.reset_index(inplace=True)
df_SumperWeekofYear["WeekofYear"].value_counts()
filt_Week35 = (df_SumperWeekofYear["WeekofYear"]==35)
df_Week35 = df_SumperWeekofYear[filt_Week35]
df_Week35

C:\Users\wasy\AppData\Local\Temp\ipykernel_18424\3044615091.py:2: FutureWarning: Series.dt.weekofyear and Series.dt.week have been deprecated. Please use Series.dt.isocalendar().week instead.
  df["WeekofYear"] = df.Date.dt.week


,Date,WeekofYear,DayofWeek,Category,Cost
52,2022-08-29,35,0,Investment,10.00
53,2022-08-29,35,0,Rent + Bills,4.94
54,2022-08-30,35,1,Groceries,43.28
55,2022-08-30,35,1,Investment,10.00
56,2022-08-31,35,2,Travel,26.15
57,2022-09-01,35,3,Rent + Bills,501.77
58,2022-09-01,35,3,Transport,63.42
59,2022-09-02,35,4,Investment,50.00
60,2022-09-03,35,5,Groceries,20.94
61,2022-09-03,35,5,Healthcare,12.50


In [40]:
#add weekday name
abr = {0: "Monday",
       1: "Tuesday",
       2: "Wednesday",
       3: "Thursday",
       4: "Friday",
       5:"Saturday",
       6:"Sunday"}

df_Week35['Weekday'] = df_Week35['DayofWeek'].replace(abr)

#sum oer day
df_sumPWeek35 = pd.DataFrame(df_Week35.groupby(["Date","DayofWeek"])["Cost"].sum().sort_index()).reset_index()
df_sumPWeek35 = df_sumPWeek35.round(2)

C:\Users\wasy\AppData\Local\Temp\ipykernel_18424\801710859.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_Week35['Weekday'] = df_Week35['DayofWeek'].replace(abr)


In [ ]:
#initializing and populating dictionary with sample week data
dict01 = {}
#add week 35
dict01["Week_35"]={}
#add week
dict01["Week_35"]["Week"] = str(df_Week35.Date.min().strftime('%d.%m.%Y'))+" - "+str(df_Week35.Date.max().strftime('%d.%m.%Y'))
dict01["Week_35"]["Week_val"] = df_sumPWeek35.iloc[:,2].values.tolist()
dict01["Week_35"]["Week_Top5"] = pd.DataFrame(df_Week35.groupby("Category")["Cost"].sum().sort_values(ascending=False).nlargest(5)).reset_index().iloc[:].values.tolist()
dict01["Week_35"]["Week_Sum"] = df_sumPWeek35.iloc[:,2].values.sum()
dict01["Week_35"]["Week_Avg"] = df_sumPWeek35.iloc[:,2].values.mean()
dict01
os.getcwd()


#store data in JSON File
with open(r'..\Data\processed_data\data_Week35.json', 'w') as fp:
    json.dump(dict01, fp)


In [41]:
#first datafile  [old!!!]
#df_Week35["Dayname"] = abr[df_Week35]
df_Week35['DayofWeek'] = df_Week35['DayofWeek'].replace(abr)
df_Week35
df_Week35.groupby("DayofWeek")["Cost"].sum().mean()
df_Week35.to_csv(r"..\Data\processed_data\dummy_weekly_data\SumperWeekofYear_Week35.csv",index=False)
df_Week35.groupby("Category")["Cost"].sum().sort_values(ascending=False).to_csv(r"..\Data\processed_data\dummy_weekly_data\SumValuesSorted_Week35.csv")
SumperWeekofYear.to_csv(r"..\Data\processed_data\dummy_weekly_data\SumperWeekofYear.csv")
df_Week35.groupby("DayofWeek")["Cost"].sum().mean()
df



C:\Users\wasy\AppData\Local\Temp\ipykernel_18424\623180731.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_Week35['DayofWeek'] = df_Week35['DayofWeek'].replace(abr)


,Category,Detail,Cost,Date,Month_period,WeekofYear,DayofWeek,Day_of_Week_Category,Month
0,Eating Out,Brunch,15.00,2022-07-25,Jul - Aug 22,30,0,Weekday,7
1,Shopping,Uniqlo,144.50,2022-07-25,Jul - Aug 22,30,0,Weekday,7
2,Hobbies,Salsa Class,130.00,2022-07-27,Jul - Aug 22,30,2,Weekday,7
3,Groceries,Groceries,6.78,2022-07-27,Jul - Aug 22,30,2,Weekday,7
4,Eating Out,Ditsch,2.00,2022-07-27,Jul - Aug 22,30,2,Weekday,7
...,...,...,...,...,...,...,...,...,...
324,Groceries,Groceries,8.26,2023-02-21,Jan - Feb 23,8,1,Weekday,2
325,Groceries,Groceries,27.25,2023-02-22,Jan - Feb 23,8,2,Weekday,2
326,Groceries,Groceries,13.69,2023-02-23,Jan - Feb 23,8,3,Weekday,2
327,Rent + Bills,Phone Norway,4.50,2023-02-24,Jan - Feb 23,8,4,Weekday,2
